In [1]:
from sagemaker import get_execution_role
role = get_execution_role()

#Bucket and Folder
bucket = 'project-b-data'
subfolder = 'proj-b/dataset-b01'

In [2]:
import boto3
conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']
#Data Set Name
for i in range(5):
    print(contents[i]['Key'])

proj-b/dataset-b01/README.txt
proj-b/dataset-b01/Tst2022-01-04LOBs.txt
proj-b/dataset-b01/Tst2022-01-04tapes.csv
proj-b/dataset-b01/Tst2022-01-05LOBs.txt
proj-b/dataset-b01/Tst2022-01-05tapes.csv


In [3]:
#Generate String Date
import pandas as pd
import time
import re
date_list = pd.date_range(start = "2022-01-04",end = "2022-04-29").tolist()
exact_date = [str(i)[0:10] for i in date_list]   #2022-01-04 format"
print(exact_date[:5])

['2022-01-04', '2022-01-05', '2022-01-06', '2022-01-07', '2022-01-08']


In [5]:
tape_csv = []

total_time = 0
for the_date in exact_date:
    t1 = time.time()
    tape_from_s3 = 's3://{}/{}'.format(bucket,subfolder+'/Tst'+the_date+'tapes.csv')

    try: 
        tape_csv.append(pd.DataFrame(pd.read_csv(tape_from_s3,header = None)))
    except:
        continue
    t2 = time.time()
    total_time += t2 - t1
    print('The {} is finished. Elapsed time is {} seconds.'.format(the_date,t2-t1))

print("Total .csv file Read time: {}".format(total_time))

The 2022-01-04 is finished. Elapsed time is 10.253955602645874 seconds.
The 2022-01-05 is finished. Elapsed time is 0.675506591796875 seconds.
The 2022-01-06 is finished. Elapsed time is 0.5000510215759277 seconds.
The 2022-01-07 is finished. Elapsed time is 0.5559003353118896 seconds.
The 2022-01-10 is finished. Elapsed time is 0.5946023464202881 seconds.
The 2022-01-11 is finished. Elapsed time is 0.6860418319702148 seconds.
The 2022-01-12 is finished. Elapsed time is 0.5395212173461914 seconds.
The 2022-01-13 is finished. Elapsed time is 0.8686573505401611 seconds.
The 2022-01-14 is finished. Elapsed time is 0.5591864585876465 seconds.
The 2022-01-17 is finished. Elapsed time is 0.5260851383209229 seconds.
The 2022-01-18 is finished. Elapsed time is 0.45144057273864746 seconds.
The 2022-01-19 is finished. Elapsed time is 0.6032276153564453 seconds.
The 2022-01-20 is finished. Elapsed time is 0.42289137840270996 seconds.
The 2022-01-21 is finished. Elapsed time is 0.5730395317077637 

In [10]:
#Concatenate all the csv file into one data frame 


full_csv_frame = pd.concat(tape_csv)
print("There are {} rows and {} columns of the full csv frame.".format(full_csv_frame.shape[0],full_csv_frame.shape[1]))
print(full_csv_frame[:5])
full_csv_frame.to_csv("Tape_csv/Full_tape.csv",index = False)

There are 5230364 rows and 11 columns of the full csv frame.
              0          1       2    3                        4   \
0  Tst2022-01-04   Exch0Lit   8.192  203   {'pool_id': 'Exch0Lit'   
1  Tst2022-01-04   Exch0Lit   8.192  199   {'pool_id': 'Exch0Lit'   
2  Tst2022-01-04   Exch0Lit  10.336  196   {'pool_id': 'Exch0Lit'   
3  Tst2022-01-04   Exch0Lit  10.672  196   {'pool_id': 'Exch0Lit'   
4  Tst2022-01-04   Exch0Lit  11.040  201   {'pool_id': 'Exch0Lit'   

                 5                6              7          8   \
0   'type': 'Trade'    'time': 8.192   'price': 203   'qty': 2   
1   'type': 'Trade'    'time': 8.192   'price': 199   'qty': 3   
2   'type': 'Trade'   'time': 10.336   'price': 196   'qty': 3   
3   'type': 'Trade'   'time': 10.672   'price': 196   'qty': 4   
4   'type': 'Trade'    'time': 11.04   'price': 201   'qty': 1   

                 9                  10  
0   'party1': 'S26'   'party2': 'B24'}  
1   'party1': 'S26'   'party2': 'B08'}  
2   

In [12]:
tape = pd.read_csv("Tape_csv/Full_tape.csv")

In [13]:
new_tape = tape.rename(columns = {"0":"Date",
                       "2":"Time","3":"Price","8":"Quantity",
                      "9":"Party1","10":"Party2"}).drop(['1',"4","5","6","7"],axis = 1)
new_tape

,Date,Time,Price,Quantity,Party1,Party2
0,Tst2022-01-04,8.192,203,'qty': 2,'party1': 'S26','party2': 'B24'}
1,Tst2022-01-04,8.192,199,'qty': 3,'party1': 'S26','party2': 'B08'}
2,Tst2022-01-04,10.336,196,'qty': 3,'party1': 'S06','party2': 'B07'}
3,Tst2022-01-04,10.672,196,'qty': 4,'party1': 'S01','party2': 'B01'}
4,Tst2022-01-04,11.040,201,'qty': 1,'party1': 'S12','party2': 'B31'}
...,...,...,...,...,...,...
5230359,Tst2022-04-29,30599.376,134,'qty': 1,'party1': 'S21','party2': 'B07'}
5230360,Tst2022-04-29,30599.520,134,'qty': 1,'party1': 'S03','party2': 'B07'}
5230361,Tst2022-04-29,30599.584,134,'qty': 1,'party1': 'S28','party2': 'B01'}
5230362,Tst2022-04-29,30599.776,134,'qty': 3,'party1': 'S04','party2': 'B01'}


In [14]:

new_tape["Date"] = list(map(lambda x:x[3:],new_tape["Date"]))
new_tape["Quantity"] = list(map(lambda x: float(x[-1]),new_tape["Quantity"]))
new_tape["Party1"] = list(map(lambda x: re.findall(r"S[\d][\d]",x)[0],new_tape["Party1"]))
new_tape["Party2"] = list(map(lambda x: re.findall(r"B[\d][\d]",x)[0],new_tape["Party2"]))
new_tape

,Date,Time,Price,Quantity,Party1,Party2
0,2022-01-04,8.192,203,2.0,S26,B24
1,2022-01-04,8.192,199,3.0,S26,B08
2,2022-01-04,10.336,196,3.0,S06,B07
3,2022-01-04,10.672,196,4.0,S01,B01
4,2022-01-04,11.040,201,1.0,S12,B31
...,...,...,...,...,...,...
5230359,2022-04-29,30599.376,134,1.0,S21,B07
5230360,2022-04-29,30599.520,134,1.0,S03,B07
5230361,2022-04-29,30599.584,134,1.0,S28,B01
5230362,2022-04-29,30599.776,134,3.0,S04,B01


In [15]:
new_tape.iloc[13]["Time"]

16.112000000000002

In [16]:
new_tape["Time"]  = new_tape["Time"].round(decimals = 3)

In [17]:
new_tape.iloc[13]["Time"]

16.112

In [18]:
new_tape.dtypes

Date         object
Time        float64
Price         int64
Quantity    float64
Party1       object
Party2       object
dtype: object

In [19]:
new_tape.to_csv("Tape_csv/Full_tape.csv",index = False)

In [20]:
pd.read_csv("Tape_csv/Full_tape.csv")

,Date,Time,Price,Quantity,Party1,Party2
0,2022-01-04,8.192,203,2.0,S26,B24
1,2022-01-04,8.192,199,3.0,S26,B08
2,2022-01-04,10.336,196,3.0,S06,B07
3,2022-01-04,10.672,196,4.0,S01,B01
4,2022-01-04,11.040,201,1.0,S12,B31
...,...,...,...,...,...,...
5230359,2022-04-29,30599.376,134,1.0,S21,B07
5230360,2022-04-29,30599.520,134,1.0,S03,B07
5230361,2022-04-29,30599.584,134,1.0,S28,B01
5230362,2022-04-29,30599.776,134,3.0,S04,B01


In [23]:
%%time
new_tape.to_csv(f"s3://project-b-data/proj-b/Tape_csv/New_Full_Tape.csv",index = False)

CPU times: user 16.3 s, sys: 378 ms, total: 16.7 s
Wall time: 20.3 s


# LOB

In [4]:
!pip install smart_open

import re
import smart_open as sm

In [5]:
import ast

In [6]:
def Read_LOB_Improved(source):
    LOB_subset_time = []
    LOB_subset_bid = []
    LOB_subset_ask = []
    time_i = -10000
    bid_i = -10000
    ask_i = -10000
    time_change_sign = False
    bid_change_sign = False
    ask_change_sign = False
    
    #Attention: errors = "ignore" for 2022-01-28
    with sm.open(source,"r",encoding="utf-8",errors = "ignore") as f:
        for i,line in enumerate(f):
            if i% 1000000 == 0:
                print(f"This is {i//1000000}th 1000000 row")
    
            #Time
            if line.strip() == '"time",':
                time_i = i 
                time_change_sign = True
                continue        
                
            if i == (time_i +1) and time_change_sign == True:
                time_number = re.findall(r"[\d]+[\.]?[\d]*",line)[0]
                #print(f"Time Number {time_number} Appended")
                time_change_sign = False
                LOB_subset_time.append(time_number)
                continue



            #Bid
            if line.strip() == '"bid",':
                bid_i = i
                bid_change_sign = True
                bid_string = ""
                continue
            if i == (bid_i + 1) and line.strip() == "[]" and bid_change_sign == True:
                #print("Bid None Appended")
                bid_change_sign = False
                LOB_subset_bid.append(None)
                continue          
            if i == (bid_i + 1) and line.strip() == "[" and bid_change_sign == True:
                bid_string += line.strip()
                continue
            elif line.strip() != "]" and bid_change_sign == True:
                bid_string += line.strip()
                continue
            elif line.strip() == "]" and bid_change_sign == True:
                bid_string += "]]"
                #Use for detect the fault line
                try:
                    #Use for 2022-01-14
                    LOB_subset_bid.append(ast.literal_eval(bid_string.strip().replace("M","").replace(" ","").replace(chr(0),"")))
                except ValueError:
                    print(f'This is {i}th row, {bid_string.strip().replace("M","").replace(" ","").replace(chr(0),"")} wrong')
                          
                except:
                    print(f'Something Wrong {i} rows')
                    
                    
                #print(f"Bid {bid_string} Appended")
                bid_change_sign = False
                continue

            #Ask
            if line.strip() == '"ask",':
                ask_i = i
                ask_change_sign = True
                ask_string = ""
                continue       
            if i == (ask_i + 1) and line.strip() == "[]" and ask_change_sign == True:
                LOB_subset_ask.append(None)
                #print("Ask None Appended")
                ask_change_sign = False
                continue       
            if i == (ask_i + 1) and line.strip() == "[" and ask_change_sign == True:
                ask_string += line.strip()
                continue
            elif line.strip() != "]" and ask_change_sign == True:
                ask_string += line.strip()
                continue
            elif line.strip() == "]" and ask_change_sign == True:
                ask_string += "]]"
                #2022-01-28
                try:
                    LOB_subset_ask.append(ast.literal_eval(ask_string.strip().replace("M","").replace(" ","").replace(chr(0),"")))
                except ValueError:
                    print(f'This is {i}th row, {ask_string.strip().replace("M","").replace(" ","").replace(chr(0),"")} wrong')
                          
                except:
                    print(f'Something Wrong {i} rows')
                                          
                #print(f"Ask {ask_string} Appended")
                ask_change_sign = False
                continue
        f.close()
        
        return [LOB_subset_time,LOB_subset_bid,LOB_subset_ask]
        
def Create_LOB_df_Improved(df_element,the_date):
    
    
    LOB_df = pd.DataFrame(df_element).T
    LOB_df['Date'] = the_date
    
    LOB_df = LOB_df.rename(columns={0: 'Time', 1: 'Bid',2: 'Ask'})
                          
    #Directly to S3
    LOB_df.to_csv(f's3://project-b-data/proj-b/LOB_csv/{the_date}-LOB.csv',index = False)
    print(f"The Shape of {the_date} LOB is {LOB_df.shape}")
    
    return LOB_df
        
        
    

In [ ]:
%%time
#The Shape of 2022-01-04 LOB is (1912500, 4)
#2022-01-04 Finished, LOB Shape: (1912500, 4)
#The 2022-01-04 is finished. Elapsed time is 524.5722577571869 seconds.


#Something Wrong 155491955 rows 2022-01-28


#"s3://project-b-data/proj-b/dataset-b01/Tst2022-01-04LOBs.txt" format
for the_date in exact_date[1:]:
    t1 = time.time()
    LOB_from_s3 = 's3://{}/{}'.format(bucket,subfolder+'/Tst'+the_date+'LOBs.txt')
    

    try:
        print(f"Start Read {the_date} LOB file.")
        Create_LOB_df_Improved(Read_LOB_Improved(LOB_from_s3),the_date)
    except:
        print(f"There is no {the_date} LOB file.")
        continue
        
    t2 = time.time()
    print('The {} is finished. Elapsed time is {} seconds.'.format(the_date,t2-t1))


Start Read 2022-01-05 LOB file.
This is 0th 1000000 row
This is 1th 1000000 row
This is 2th 1000000 row
This is 3th 1000000 row
This is 4th 1000000 row
This is 5th 1000000 row
This is 6th 1000000 row
This is 7th 1000000 row
This is 8th 1000000 row
This is 9th 1000000 row
This is 10th 1000000 row
This is 11th 1000000 row
This is 12th 1000000 row
This is 13th 1000000 row
This is 14th 1000000 row
This is 15th 1000000 row
This is 16th 1000000 row
This is 17th 1000000 row
This is 18th 1000000 row
This is 19th 1000000 row
This is 20th 1000000 row
This is 21th 1000000 row
This is 22th 1000000 row
This is 23th 1000000 row
This is 24th 1000000 row
This is 25th 1000000 row
This is 26th 1000000 row
This is 27th 1000000 row
This is 28th 1000000 row
This is 29th 1000000 row
This is 30th 1000000 row
This is 31th 1000000 row
This is 32th 1000000 row
This is 33th 1000000 row
This is 34th 1000000 row
This is 35th 1000000 row
This is 36th 1000000 row
This is 37th 1000000 row
This is 38th 1000000 row
Thi

In [7]:
pd.read_csv(f"s3://project-b-data/proj-b/LOB_csv/2022-01-04-LOB.csv")

,Time,Bid,Ask,Date
0,0.000,NaN,NaN,2022-01-04
1,0.016,NaN,NaN,2022-01-04
2,0.032,NaN,NaN,2022-01-04
3,0.048,NaN,NaN,2022-01-04
4,0.064,NaN,NaN,2022-01-04
...,...,...,...,...
1912495,30599.920,"[[220, 4], [211, 1], [175, 4], [170, 5], [161,...","[[236, 2], [411, 5], [436, 1], [659, 1], [725,...",2022-01-04
1912496,30599.936,"[[220, 4], [211, 1], [175, 4], [170, 5], [161,...","[[236, 2], [411, 5], [436, 1], [659, 1], [725,...",2022-01-04
1912497,30599.952,"[[220, 4], [211, 1], [175, 4], [170, 5], [161,...","[[236, 2], [411, 5], [436, 1], [659, 1], [725,...",2022-01-04
1912498,30599.968,"[[220, 4], [211, 1], [175, 4], [170, 5], [161,...","[[236, 2], [411, 5], [436, 1], [611, 1], [725,...",2022-01-04


In [8]:
pd.read_csv(f"s3://project-b-data/proj-b/LOB_csv/2022-01-14-LOB.csv")

,Time,Bid,Ask,Date
0,0.000,NaN,NaN,2022-01-14
1,0.016,NaN,NaN,2022-01-14
2,0.032,NaN,NaN,2022-01-14
3,0.048,NaN,NaN,2022-01-14
4,0.064,NaN,NaN,2022-01-14
...,...,...,...,...
1912487,30599.920,"[[227, 2], [225, 2], [221, 3], [188, 1], [184,...","[[230, 2], [418, 4], [428, 4], [456, 4], [534,...",2022-01-14
1912488,30599.936,"[[227, 2], [225, 2], [221, 3], [188, 1], [184,...","[[230, 2], [418, 4], [428, 4], [456, 4], [534,...",2022-01-14
1912489,30599.952,"[[227, 2], [225, 2], [221, 3], [188, 1], [184,...","[[230, 2], [418, 4], [428, 4], [456, 4], [534,...",2022-01-14
1912490,30599.968,"[[227, 2], [225, 2], [221, 3], [188, 1], [184,...","[[230, 2], [418, 4], [428, 4], [456, 4], [534,...",2022-01-14


In [9]:
pd.read_csv(f"s3://project-b-data/proj-b/LOB_csv/2022-01-28-LOB.csv")

,Time,Bid,Ask,Date
0,0.000,NaN,NaN,2022-01-28
1,0.016,NaN,NaN,2022-01-28
2,0.032,NaN,NaN,2022-01-28
3,0.048,NaN,NaN,2022-01-28
4,0.064,NaN,NaN,2022-01-28
...,...,...,...,...
1912487,30599.920,"[[202, 3], [199, 3], [188, 4], [173, 7], [154,...","[[213, 4], [219, 1], [220, 5], [330, 2], [377,...",2022-01-28
1912488,30599.936,"[[202, 3], [199, 3], [188, 4], [173, 7], [154,...","[[213, 4], [219, 1], [220, 5], [330, 2], [377,...",2022-01-28
1912489,30599.952,"[[202, 3], [199, 3], [188, 4], [173, 4], [154,...","[[213, 4], [219, 1], [220, 5], [330, 2], [377,...",2022-01-28
1912490,30599.968,"[[202, 3], [199, 3], [188, 4], [173, 4], [154,...","[[213, 4], [219, 1], [220, 5], [330, 2], [377,...",2022-01-28


In [10]:
exact_date[25:]

['2022-01-29',
 '2022-01-30',
 '2022-01-31',
 '2022-02-01',
 '2022-02-02',
 '2022-02-03',
 '2022-02-04',
 '2022-02-05',
 '2022-02-06',
 '2022-02-07',
 '2022-02-08',
 '2022-02-09',
 '2022-02-10',
 '2022-02-11',
 '2022-02-12',
 '2022-02-13',
 '2022-02-14',
 '2022-02-15',
 '2022-02-16',
 '2022-02-17',
 '2022-02-18',
 '2022-02-19',
 '2022-02-20',
 '2022-02-21',
 '2022-02-22',
 '2022-02-23',
 '2022-02-24',
 '2022-02-25',
 '2022-02-26',
 '2022-02-27',
 '2022-02-28',
 '2022-03-01',
 '2022-03-02',
 '2022-03-03',
 '2022-03-04',
 '2022-03-05',
 '2022-03-06',
 '2022-03-07',
 '2022-03-08',
 '2022-03-09',
 '2022-03-10',
 '2022-03-11',
 '2022-03-12',
 '2022-03-13',
 '2022-03-14',
 '2022-03-15',
 '2022-03-16',
 '2022-03-17',
 '2022-03-18',
 '2022-03-19',
 '2022-03-20',
 '2022-03-21',
 '2022-03-22',
 '2022-03-23',
 '2022-03-24',
 '2022-03-25',
 '2022-03-26',
 '2022-03-27',
 '2022-03-28',
 '2022-03-29',
 '2022-03-30',
 '2022-03-31',
 '2022-04-01',
 '2022-04-02',
 '2022-04-03',
 '2022-04-04',
 '2022-04-

In [ ]:
%%time
#The Shape of 2022-01-04 LOB is (1912500, 4)
#2022-01-04 Finished, LOB Shape: (1912500, 4)
#The 2022-01-04 is finished. Elapsed time is 524.5722577571869 seconds.





#"s3://project-b-data/proj-b/dataset-b01/Tst2022-01-04LOBs.txt" format
for the_date in exact_date[25:]:
    t1 = time.time()
    LOB_from_s3 = 's3://{}/{}'.format(bucket,subfolder+'/Tst'+the_date+'LOBs.txt')
    

    try:
        print(f"Start Read {the_date} LOB file.")
        Create_LOB_df_Improved(Read_LOB_Improved(LOB_from_s3),the_date)
    except:
        print(f"There is no {the_date} LOB file.")
        continue
        
    t2 = time.time()
    print('The {} is finished. Elapsed time is {} seconds.'.format(the_date,t2-t1))

Start Read 2022-01-29 LOB file.
There is no 2022-01-29 LOB file.
Start Read 2022-01-30 LOB file.
There is no 2022-01-30 LOB file.
Start Read 2022-01-31 LOB file.
This is 0th 1000000 row
This is 1th 1000000 row
This is 2th 1000000 row
This is 3th 1000000 row
This is 4th 1000000 row
This is 5th 1000000 row
This is 6th 1000000 row
This is 7th 1000000 row
This is 8th 1000000 row
This is 9th 1000000 row
This is 10th 1000000 row
This is 11th 1000000 row
This is 12th 1000000 row
This is 13th 1000000 row
This is 14th 1000000 row
This is 15th 1000000 row
This is 16th 1000000 row
This is 17th 1000000 row
This is 18th 1000000 row
This is 19th 1000000 row
This is 20th 1000000 row
This is 21th 1000000 row
This is 22th 1000000 row
This is 23th 1000000 row
This is 24th 1000000 row
This is 25th 1000000 row
This is 26th 1000000 row
This is 27th 1000000 row
This is 28th 1000000 row
This is 29th 1000000 row
This is 30th 1000000 row
This is 31th 1000000 row
This is 32th 1000000 row
This is 33th 1000000 ro